<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
Lokale und Open Source Modelle
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import (
    check_environment,
    get_ipinfo,
    setup_api_keys,
    mprint,
    install_packages,
    mermaid,
    get_model_profile,
    extract_thinking,
    load_chat_prompt_template
)
print()
check_environment()
print()
get_ipinfo()

<p><font color='black' size="5">
⏸️ 5-Minuten-Check:
</font></p>

**Ziel:** Prüfen, ob du das vorherige Kapitel verstanden hast – nicht, ob es gerade läuft.

**Aufgabe** (5 Minuten, ohne Vorlage):

Rekonstruiere die zentrale Idee oder Code-Struktur des letzten Abschnitts selbstständig
(kein Copy & Paste, kein Nachschlagen).

Wähle eine der folgenden Optionen:

+ Erkläre in 1–2 Sätzen, was hier konzeptionell passiert.

+ Verändere eine Kleinigkeit (z. B. Prompt, Parameter, Reihenfolge) und beschreibe die Auswirkung.

+ Markiere eine Stelle, die du nicht sicher erklären kannst, und formuliere eine konkrete Frage dazu.

**Hinweis:**
Nicht alles muss „fertig“ oder „korrekt“ sein. Entscheidend ist, wo dein Verständnis gerade endet

# 1 | Einführung
---


Lokale und Open Source Modelle bieten eine Alternative zu Cloud-basierten KI-Diensten wie OpenAI. Im Gegensatz zu API-basierten Diensten werden diese Modelle direkt auf dem lokalen System ausgeführt, was Vorteile in Bezug auf Datenschutz, Kosten und Anpassungsmöglichkeiten bietet.



<p><font color='black' size="5">
Vor- und Nachteile lokaler Modelle
</font></p>

- **Vorteile**: Datenschutz, keine API-Kosten, keine Internetverbindung erforderlich, volle Kontrolle
- **Nachteile**: Höhere Hardwareanforderungen, begrenzte Modellleistung auf Standard-Hardware, Installations- und Wartungsaufwand



<p><font color='black' size="5">
Anwendungsbereiche lokaler Modelle
</font></p>

- Entwicklung und Prototyping
- Datenschutzkritische Anwendungen
- Offline-Anwendungen
- Kostensensitive Projekte



# 2 | Ollama
---

<img src="https://ollama.com/public/ollama.png" class="logo" width="120"/>





Ollama ist ein Tool, das die einfache Installation und Nutzung von Open Source LLMs auf dem eigenen Computer ermöglicht. Es bietet eine benutzerfreundliche Schnittstelle zum Herunterladen, Verwalten und Ausführen verschiedener Sprachmodelle.

[Ollama](https://ollama.com/)

In [ ]:
#@title 🧜‍♀️ Prozess-Diagramm { display-mode: "form" }

diagram = """
graph TD
    subgraph Colab ["<b>Google Colab Runtime</b>"]
        direction TB

        Code["Python Code<br/><small>Notebook-Zelle</small>"]

        subgraph Backend ["Hintergrund"]
            Ollama["Ollama Server<br/><small>localhost:11434</small>"]
            Models["Lokale Modelle"]
        end

        GPU["GPU<br/><small>T4 / A100</small>"]
    end

    subgraph Extern ["Extern"]
        Registry["Ollama Registry"]
    end

    %% Haupt-Workflow (nummeriert)
    Code -->|"#1 API Call"| Ollama
    Ollama -->|"#2 laden"| Models
    Ollama -->|"#3 Inferenz"| GPU
    GPU -->|"#4 Ergebnis"| Ollama
    Ollama -->|"#5 Response"| Code

    %% Setup (gestrichelt)
    Registry -.->|"ollama pull"| Models

    %% Styling
    style Colab fill:#f8f9fa,stroke:#495057,stroke-width:2px
    style GPU fill:#d4edda,stroke:#28a745,stroke-width:2px
    style Ollama fill:#e3f2fd,stroke:#1976d2,stroke-width:2px
    style Registry fill:#fff3cd,stroke:#ffc107
"""
mermaid(diagram)


<p><font color='black' size="5">
Installation von Ollama
</font></p>

In [ ]:
# Installation in Linux/Colab
!sudo apt update
!sudo apt install -y pciutils
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh

<p><font color='black' size="5">
Starten des Ollama-Servers
</font></p>

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(10)  # Kurz warten, bis der Server hochgefahren ist

<p><font color='black' size="5">
Modelle herunterladen und verwalten
</font></p>

In [ ]:
# Modelle herunterladen
!ollama pull llama3

In [ ]:
# Modelle herunterladen
!ollama pull gemma3

In [ ]:
# Verfügbare Modelle anzeigen
!ollama list

# 3 | Lokale Modelle und LangChain
---

LangChain bietet eine nahtlose Integration mit Ollama, wodurch lokale Modelle in komplexen Anwendungen eingesetzt werden können.



<p><font color='black' size="5">
Einfache Verwendung von Ollama mit LangChain
</font></p>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chat_models import init_chat_model

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller KI-Assistent. Antworte sachlich und kurz."),
    ("human", "Frage: {question}\n\nAntwort: Denke Schritt für Schritt.")
])

# Model
llm = init_chat_model("ollama:llama3", temperature=0.1)

# Parser
parser = StrOutputParser()

# Chain zusammensetzen und ausführen
chain = prompt | llm | parser

<p><font color='black' size="5">
Beispiele Llama 3
</font></p>

In [ ]:
# Beispiel 1: Gesundheitsfrage
response = chain.invoke({"question": "Was ist der einfachste Weg Gewicht abzunehmen?"})
mprint("## 🤖 Local LLM")
mprint(response)

In [ ]:
# Beispiel 2: Allgemeinwissen
response = chain.invoke({"question": "Wer war Albert Einstein?"})
mprint("## 🤖 Local LLM")
mprint(response)

<p><font color='black' size="5">
Beispiele Gemma 3
</font></p>

In [ ]:
# Model
llm = init_chat_model("ollama:gemma3", temperature=0.1)

# Chain zusammensetzen und ausführen
chain = prompt | llm | parser

In [ ]:
# Beispiel 1: Gesundheitsfrage
response = chain.invoke({"question": "Was ist der einfachste Weg Gewicht abzunehmen?"})
mprint("## 🤖 Local LLM")
mprint(response)

In [ ]:
# Beispiel 2: Allgemeinwissen
response = chain.invoke({"question": "Wer war Albert Einstein?"})
mprint("## 🤖 Local LLM")
mprint(response)


# 4 | Open Source vs. Closed Source
---

<p><font color='black' size="5">
Open Source Modelle
</font></p>



Open Source KI-Modelle zeichnen sich durch ihre frei zugänglichen Gewichte und Architekturen aus. Diese Modelle werden unter Lizenzen wie MIT, Apache oder Creative Commons veröffentlicht, die es Nutzern erlauben, den Code einzusehen, zu modifizieren und für eigene Zwecke anzupassen.

**Vorteile von Open Source Modellen:**

- **Transparenz**: Einsicht in die Funktionsweise und Trainingsmethoden
- **Anpassbarkeit**: Möglichkeit zum Fine-Tuning für spezifische Anwendungsfälle
- **Keine API-Kosten**: Nach dem Download entstehen keine laufenden Nutzungskosten
- **Datenschutz**: Vollständige Kontrolle über die Datenverarbeitung
- **Community-Unterstützung**: Regelmäßige Verbesserungen durch die Community

**Herausforderungen:**

- **Hardwareanforderungen**: Größere Modelle benötigen leistungsstarke Hardware
- **Expertise**: Technisches Know-how für Installation und Optimierung nötig
- **Qualitätsunterschiede**: Leistung kann bei gleicher Größe hinter kommerziellen Modellen zurückbleiben



<p><font color='black' size="5">
Closed Source Modelle
</font></p>

Closed Source Modelle wie GPT-4 von OpenAI oder Claude von Anthropic werden nur über APIs angeboten, ohne Zugriff auf die zugrunde liegenden Modellgewichte.

**Vorteile von Closed Source Modellen:**

- **Leistungsstärke**: Oft fortschrittlicher als vergleichbare Open Source Alternativen
- **Einfache Nutzung**: Keine lokale Installation oder Wartung nötig
- **Regelmäßige Updates**: Automatische Verbesserungen ohne eigenen Aufwand
- **Skalierbarkeit**: Flexibel an unterschiedlichen Bedarf anpassbar

**Nachteile:**

- **Kosten**: Nutzungsbasierte Gebühren
- **Abhängigkeit**: Angewiesenheit auf externe Dienste und deren Verfügbarkeit
- **Datenschutzbedenken**: Daten verlassen das eigene System
- **Eingeschränkte Anpassbarkeit**: Beschränkte Möglichkeiten zur Modifikation



# 5 | Beispiele Open Source Modelle
---

<p><font color='black' size="5">
Phi-4 (Microsoft)
</font></p>

Microsoft's Phi-4 ist ein kompaktes, aber leistungsstarkes Modell, das trotz seiner geringen Größe beeindruckende Ergebnisse liefert.

**Technische Details:**

- Verfügbar in verschiedenen Größen (2,7B bis 13B Parameter)
- Optimiert für das Ausführen auf durchschnittlicher Hardware
- Besonders stark in Coding-Aufgaben und logischem Denken
- Trainiert mit hochwertigen, kuratierten Daten
- Verwendet eine optimierte Transformerarchitektur

**Einsatzbereiche:**

- Softwareentwicklung und Code-Generierung
- Textgenerierung mit geringen Hardwareanforderungen
- Bildung und Wissensanwendungen



<p><font color='black' size="5">
Gemma (Google)
</font></p>

Google's Gemma ist eine Familie leichtgewichtiger, offener Modelle, die speziell für Effizienz optimiert wurden.

**Technische Details:**

- Größen: 2B und 7B Parameter
- Entwickelt mit einem starken Fokus auf verantwortungsvolle KI
- Basiert auf der Gemini-Modellfamilie
- Unterstützt Kontextfenster bis zu 8K Tokens
- Verfügbar in verschiedenen Quantisierungen (GGUF)

**Besonderheiten:**

- Starke Leistung in mehrsprachigen Anwendungen
- Effiziente Ressourcennutzung
- Gute Balance zwischen Modellgröße und Fähigkeiten



<p><font color='black' size="5">
Llama 2 und Llama 3 (Meta)
</font></p>

Meta's Llama-Serie hat die Open-Source-LLM-Landschaft revolutioniert und bietet leistungsstarke Modelle mit differenzierten Lizenzen.

**Technische Details:**

- Llama 3: Verfügbar in 8B und 70B Varianten, mit verbesserter Multitask-Leistung
- Llama 2: 7B, 13B und 70B Parameter
- Kontextfenster: Bis zu 4K Tokens (Llama 2), 8K+ (Llama 3)
- Chat-optimierte Varianten verfügbar

**Besonderheiten:**

- Starke Leistung in Reasoning-Aufgaben
- Verbesserte Instruktionsbefolgung in neueren Versionen
- Semi-offene Lizenz mit Einschränkungen für kommerzielle Nutzung ab bestimmten Nutzer-Schwellenwerten



<p><font color='black' size="5">
Mistral (Mistral AI)
</font></p>

Das französische Startup Mistral AI hat mit seinen effizienten Modellen für Aufsehen gesorgt.

**Technische Details:**

- Mistral 7B: Beeindruckende Leistung trotz kompakter Größe
- Mixtral 8x7B: Ein Mixture-of-Experts (MoE) Modell mit starker Leistung
- Verbesserte Architekturen mit effizienten Attention-Mechanismen
- Unterstützung für bis zu 32K Token Kontextfenster

**Besonderheiten:**

- Herausragende Leistung für die Modellgröße
- Spezialisierte Varianten für Code-Generierung (Codestral)
- Moderne Architekturoptimierungen für bessere Effizienz



<p><font color='black' size="5">
Falcon (Technology Innovation Institute)
</font></p>

Die Falcon-Modelle wurden vom Technology Innovation Institute in den VAE entwickelt und bieten eine großzügige Lizenz für kommerzielle Anwendungen.

**Technische Details:**

- Größen: 1B, 7B und 40B Parameter
- Trainiert auf einem großen, diversen Datensatz
- Optimierte Transformer-Architektur für Effizienz

**Besonderheiten:**

- Permissive Lizenz für kommerzielle Anwendungen
- Gute Mehrsprachigkeit
- Hohe Qualität in allgemeinen Wissensabfragen




# 6 | Lizenzierung und rechtliche Aspekte
---

Die Lizenzen von Open Source Modellen unterscheiden sich erheblich und beeinflussen deren Einsatzmöglichkeiten:

**MIT Lizenz (z. B. OpenLLaMA, GPT-NeoX)**
- Sehr liberal, erlaubt kommerzielle Nutzung ohne Einschränkungen  
- Erfordert lediglich die Nennung des ursprünglichen Urhebers  
- Keine Weitergabelizenzverpflichtung (nicht copyleft)  
- Einfach und weit verbreitet in Open-Source-Projekten  

**Apache 2.0 (z. B. Gemma, Falcon)**
- Sehr liberal, erlaubt kommerzielle Nutzung ohne starke Einschränkungen  
- Erfordert Hinweis auf ursprüngliches Urheberrecht  
- Keine Weitergabelizenzverpflichtung (nicht copyleft)  
- Enthält eine Klausel zum Schutz vor Patentstreitigkeiten  

**Meta Llama Lizenz**
- Erlaubt kommerzielle Nutzung mit Einschränkungen (z. B. Nutzerschwellen)  
- Enthält spezifische Bedingungen für verantwortungsvolle Nutzung  
- Verbietet bestimmte Anwendungsfälle wie Desinformation oder illegale Aktivitäten  
- Nicht so frei wie Apache oder MIT, aber offener als proprietäre Modelle  

**Mistral AI Lizenz**
- Erlaubt kommerzielle und nicht-kommerzielle Nutzung  
- Enthält ethische Nutzungsbeschränkungen  
- Unterschiedliche Bedingungen für verschiedene Modellversionen  
- Im Allgemeinen offener als die Meta Llama Lizenz  

**Creative Commons (z. B. BLOOM unter CC-BY-SA)**
- Verschiedene Varianten mit unterschiedlichem Freiheitsgrad  
- **CC-BY**: Erlaubt kommerzielle Nutzung, solange der Urheber genannt wird  
- **CC-BY-SA**: Weitergabe nur unter gleichen Lizenzbedingungen erlaubt (copyleft)  
- **CC-BY-NC**: Keine kommerzielle Nutzung erlaubt  
- **CC-BY-ND**: Keine Bearbeitung oder Ableitung erlaubt  
- Häufig für Open-Access-Modelle genutzt, aber mit Einschränkungen je nach Variante  

<br>

**Disclaimer**    
Die oben aufgeführten Informationen zu Lizenzmodellen dienen ausschließlich allgemeinen Informationszwecken und stellen keine rechtliche Beratung dar. Die Bedingungen einzelner Lizenzen können sich ändern, und ihre Auslegung kann von der jeweiligen Rechtsprechung abhängen.

Vor der Nutzung oder Weitergabe eines KI-Modells sollten die offiziellen Lizenzdokumente sorgfältig geprüft und bei Unsicherheiten eine juristische Beratung eingeholt werden. Wir übernehmen keine Haftung für die Vollständigkeit, Richtigkeit oder Anwendbarkeit der bereitgestellten Informationen.



# 7 | Auswahlkriterien Open Source
---

Bei der Wahl eines geeigneten Open Source Modells sollten folgende Faktoren berücksichtigt werden:

**Hardwareanforderungen:**

- Größere Modelle (40B+): Benötigen High-End-GPUs mit 24GB+ VRAM
- Mittlere Modelle (7-13B): Laufen auf Consumer-GPUs mit 8-16GB VRAM
- Kleine Modelle (1-2B): Können auf Standard-Laptops oder CPUs ausgeführt werden
- Quantisierung (INT4, INT8) reduziert Hardwareanforderungen erheblich

**Anwendungsfall:**

- Allgemeine Textgenerierung: Llama 3, Mistral und Gemma bieten gute Allround-Leistung
- Code-Generierung: Phi, CodeLlama oder Codestral für spezialisierte Coding-Fähigkeiten
- Wissenschaftliche Aufgaben: Spezialisierte Domänenmodelle bevorzugen
- Mehrsprachigkeit: Auf Modelle mit breitem Sprachtraining (z.B. BLOOM) setzen

**Kontextfenster:**

- Standard (4-8K Tokens): Für die meisten einfachen Anwendungen ausreichend
- Erweitert (16-32K Tokens): Für Dokumentenanalyse und komplexe Aufgaben
- Lang (64K+ Tokens): Für die Verarbeitung ganzer Dokumente oder Codebases



# 8 | Zukunftstrends bei Open Source
---



Die Open Source LLM-Landschaft entwickelt sich rapide weiter. Folgende Trends sind zu beobachten:

**Effizienzsteigerungen:**

- Kleinere Modelle mit der Leistung größerer Vorgänger
- Verbesserte Quantisierungstechniken für schnellere Inferenz
- Optimierte Architekturen (MoE, FlashAttention, etc.)

**Spezialisierung:**

- Domänenspezifische Modelle für Medizin, Recht, Finanzen, etc.
- Aufgabenspezifische Varianten für Code, Mathematik, Reasoning
- Sprachspezifische Modelle für nicht-englische Sprachen

**Verbesserte Lizenzen:**

- Entwicklung ausgewogenerer Lizenzmodelle
- Klarere Richtlinien für verantwortungsvolle KI-Nutzung
- Industriefreundlichere Bedingungen für kommerzielle Anwendungen

**Multimodalität:**

- Integration von Text- und Bildverständnis
- Erweiterung auf Audio- und Videoverarbeitung
- Verbesserte Integration mit externen Tools und Wissensquellen

Diese Entwicklungen werden die Lücke zwischen Open Source und proprietären Modellen weiter verringern und die Einstiegshürden für lokale KI-Anwendungen senken.



<p><font color='black' size="5">
Modellperformance
</font></p>

![My Image](https://raw.githubusercontent.com/ralf-42/GenAI/main/07_image/top-mmlu-open-closed-banner_klein.png)

[Ben Cottier et al. (2024), "How Far Behind Are Open Models?". Published online at epoch.ai.](https://epoch.ai/blog/open-models-report)

# A | Aufgabe


Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Vergleich von Open Source Modellen
</font></p>

Installieren Sie Ollama auf Ihrem lokalen System und vergleichen Sie die Leistung von mindestens zwei verschiedenen Modellen (z.B. Llama2 und Gemma) bei der Beantwortung derselben Prompts.

Vergleichen Sie die Antworten der Modelle. Kriterien: Umfang, Qualität, Antwortzeit, ...


<p><font color='black' size="5">
Chat-Anwendung mit lokalem LLM
</font></p>


Entwickeln Sie eine einfache Gradio-basierte Chat-Anwendung, die ein lokales Modell über Ollama nutzt. Die Anwendung soll:

```python
import gradio as gr
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage
import os

# Ollama-LLM initialisieren
llm = OllamaLLM(model="gemma", temperature=0.7)

# Chat-Historie
chat_verlauf = []

def chat_antwort(nachricht, verlauf):
    """
    Verarbeitet die Nutzereingabe und generiert eine Antwort vom LLM
    """
    # Aktuelle Nachricht zur Historie hinzufügen
    if nachricht:
        verlauf.append((nachricht, ""))
        
        # Chat-Kontext erstellen (begrenzt auf die letzten 5 Nachrichten für Effizienz)
        kontext = ""
        for i in range(max(0, len(verlauf)-5), len(verlauf)):
            if i < len(verlauf)-1:  # Alle vorherigen Nachrichten
                kontext += f"Mensch: {verlauf[i][0]}\nAssistent: {verlauf[i][1]}\n"
            else:  # Aktuelle Nachricht
                kontext += f"Mensch: {verlauf[i][0]}\nAssistent: "
        
        # Antwort vom Modell generieren
        antwort = llm.invoke(kontext)
        
        # Antwort zur Historie hinzufügen
        verlauf[-1] = (nachricht, antwort)
    
    return "", verlauf

# Gradio-Interface erstellen
with gr.Blocks(title="Lokaler KI-Assistent") as demo:
    gr.Markdown("# Chat mit einem lokalen KI-Modell")
    gr.Markdown("Dieser Chat verwendet das Gemma-Modell über Ollama")
    
    chatbot = gr.Chatbot(label="Chat-Verlauf")
    msg = gr.Textbox(label="Ihre Nachricht", placeholder="Schreiben Sie hier Ihre Nachricht...")
    
    msg.submit(chat_antwort, [msg, chatbot], [msg, chatbot])
    
    gr.Markdown("### Hinweise zur Nutzung:")
    gr.Markdown("- Die Antworten werden lokal auf Ihrem System generiert")
    gr.Markdown("- Keine Daten werden an externe Server gesendet")
    gr.Markdown("- Die Antwortgeschwindigkeit hängt von Ihrer Hardware ab")

# Anwendung starten
demo.launch()
```

Testen & erweitern Sie die Anwendung:
1. Testen Sie den Code, korrigieren Sie etwaige Fehler.
2. Modellauswahl (zwischen verschiedenen installierten Ollama-Modellen)
3. Parameter-Einstellungen (Temperatur, Top-K, Top-P)
4. Speichern und Laden von Chat-Verläufen
5. Möglichkeit, die generierten Antworten zu bewerten (Daumen hoch/runter)

<p><font color='black' size="5">
RAG-System mit lokalem LLM
</font></p>



Entwickeln Sie ein einfaches Retrieval-Augmented Generation (RAG) System, das lokale PDF-Dokumente einliest und die Beantwortung von Fragen mit einem lokalen Modell ermöglicht:

```python
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
import gradio as gr
import os

# Modell initialisieren
llm = OllamaLLM(model="llama2", temperature=0.1)

# Embedding-Modell laden (lokal, ohne API-Calls)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vektorstore für Dokumente
vector_store = None

def dokumente_laden(dateien):
    """Lädt PDF-Dokumente und erstellt eine Vektordatenbank"""
    global vector_store
    
    dokumente = []
    for datei in dateien:
        loader = PyPDFLoader(datei.name)
        dokumente.extend(loader.load())
    
    # Dokumente in kleinere Chunks aufteilen
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_documents(dokumente)
    
    # Vektordatenbank erstellen
    vector_store = FAISS.from_documents(chunks, embeddings)
    
    return f"{len(chunks)} Textabschnitte aus {len(dateien)} Dokumenten geladen"

def frage_beantworten(frage):
    """Beantwortet eine Frage basierend auf den geladenen Dokumenten"""
    if vector_store is None:
        return "Bitte laden Sie zuerst Dokumente hoch."
    
    # Relevante Dokumente abrufen
    docs = vector_store.similarity_search(frage, k=3)
    
    # Kontext erstellen
    kontext = "\n\n".join([doc.page_content for doc in docs])
    
    # Prompt erstellen
    prompt_vorlage = """
    Du bist ein hilfreicher Assistent, der Fragen basierend auf gegebenen Informationen beantwortet.
    
    Kontext:
    {kontext}
    
    Frage: {frage}
    
    Deine Antwort sollte sich auf die im Kontext gegebenen Informationen beschränken.
    Wenn du keine Antwort im Kontext findest, sage ehrlich, dass du es nicht weißt.
    """
    
    prompt = PromptTemplate(
        template=prompt_vorlage,
        input_variables=["kontext", "frage"]
    )
    
    # Antwort vom LLM bekommen
    antwort = llm.invoke(prompt.format(kontext=kontext, frage=frage))
    
    return antwort

# Gradio-Interface
with gr.Blocks() as demo:
    gr.Markdown("# Lokales Frage-Antwort-System mit RAG")
    
    with gr.Tab("Dokumente laden"):
        file_input = gr.File(file_count="multiple", label="PDF-Dokumente")
        upload_button = gr.Button("Dokumente laden")
        upload_output = gr.Textbox(label="Status")
        upload_button.click(dokumente_laden, inputs=[file_input], outputs=[upload_output])
    
    with gr.Tab("Fragen stellen"):
        question_input = gr.Textbox(label="Ihre Frage zum Dokument")
        answer_button = gr.Button("Frage stellen")
        answer_output = gr.Textbox(label="Antwort")
        answer_button.click(frage_beantworten, inputs=[question_input], outputs=[answer_output])

demo.launch()
```

Testen und erweitern Sie das System:
1. Testen & ggf. korrigieren des Code
2. Unterstützung für weitere Dateiformate (DOCX, TXT, HTML)
3. Möglichkeit, die Qualität der Antworten zu verbessern durch:
   - Modellauswahl
   - Parameteroptimierung
   - Verbesserung des Prompts
4. Implementierung einer Möglichkeit zur Quellennachverfolgung (aus welchem Teil des Dokuments stammt die Information)
5. Visualisierung der relevanten Textpassagen im Dokument